Initial data collection and exploration to get familiar with the data...

In [ ]:
#! wget -P ../../data/raw https://aqua.kingcounty.gov/extranet/assessor/Real%20Property%20Sales.zip
#! unzip ../../data/raw/Real\ Property\ Sales.zip
#! wget -P ../../data/raw https://aqua.kingcounty.gov/extranet/assessor/Residential%20Building.zip
#! unzip ../../data/raw/Residential\ Building.zip
#! wget -P ../../data/raw https://aqua.kingcounty.gov/extranet/assessor/Parcel.zip
#! unzip ../../data/raw/Parcel.zip
#! wget -P ../../data/raw https://aqua.kingcounty.gov/extranet/assessor/Lookup.zip
#! unzip ../../data/raw/Lookup.zip

In [2]:
import pandas as pd
import datetime as dt
Real_Prop = pd.read_csv("../EXTR_RPSale.csv")
Res_Bldg = pd.read_csv("../EXTR_ResBldg.csv")
Parcel = pd.read_csv("../EXTR_Parcel.csv", encoding='latin-1')
codes = pd.read_csv('../EXTR_LookUp.csv')

/opt/anaconda3/envs/oy-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/envs/oy-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Only interested in 2019 sales ###

In [3]:
Real_Prop['DocumentDate'] = pd.to_datetime(Real_Prop['DocumentDate'])
Real_Prop19 = Real_Prop[Real_Prop['DocumentDate'].dt.year == 2019]
Real_Prop19

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
4,3027422,213043,120,2019-12-20,560000,20191226000848,,,,,...,11,6,3,N,N,N,N,1,8,
118,2999169,919715,200,2019-07-08,192000,20190712001080,,,,,...,3,2,3,N,N,N,N,1,3,
144,3000673,894444,200,2019-06-26,185000,20190722001395,,,,,...,3,2,3,N,N,N,N,1,3,
164,3002257,940652,630,2019-07-22,435000,20190730001339,,,,,...,11,6,3,N,N,N,N,1,8,
445,2980836,937630,695,2019-03-28,550000,20190404001008,,,,,...,3,6,3,N,N,N,N,1,8,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2089056,3004408,66000,2210,2019-08-07,41040000,20190812000849,,,,,...,3,7,22,N,N,N,N,18,2,45
2089057,3004408,66000,2225,2019-08-07,41040000,20190812000849,,,,,...,3,7,22,N,N,N,N,18,2,45
2089058,3004408,66000,2195,2019-08-07,41040000,20190812000849,,,,,...,3,7,22,N,N,N,N,18,2,45
2089059,3004408,66000,2220,2019-08-07,41040000,20190812000849,,,,,...,3,7,22,N,N,N,N,18,2,45


### Cleaning up column names ###

In [4]:
colnames = Real_Prop19.columns
colnames = [x.lower() for x in colnames]
Real_Prop19.columns = colnames

In [5]:
Real_Prop19.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61363 entries, 4 to 2089082
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   excisetaxnbr        61363 non-null  int64         
 1   major               61363 non-null  object        
 2   minor               61363 non-null  object        
 3   documentdate        61363 non-null  datetime64[ns]
 4   saleprice           61363 non-null  int64         
 5   recordingnbr        61363 non-null  object        
 6   volume              61363 non-null  object        
 7   page                61363 non-null  object        
 8   platnbr             61363 non-null  object        
 9   plattype            61363 non-null  object        
 10  platlot             61363 non-null  object        
 11  platblock           61363 non-null  object        
 12  sellername          61363 non-null  object        
 13  buyername           61363 non-null  object  

### Padding up major and minor to bring them together to uniquely identify entries ###

In [6]:
Real_Prop19['major'] = Real_Prop19['major'].astype(str)

<ipython-input-6-8bd2fe129560>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Real_Prop19['major'] = Real_Prop19['major'].astype(str)


In [7]:
def major_padded(row):
    if len(row) == 6:
        return row
    elif len(row) == 5:
        return '0'+row
    elif len(row) == 4:
        return '00'+row
    elif len(row) == 3:
        return '000'+row
    elif len(row) == 2:
        return '0000'+row
    elif len(row) == 1:
        return '00000'+row
    else:
        return '000000'
Real_Prop19['major'] = Real_Prop19.major.apply(major_padded)
Real_Prop19

<ipython-input-7-0aa252928095>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Real_Prop19['major'] = Real_Prop19.major.apply(major_padded)


,excisetaxnbr,major,minor,documentdate,saleprice,recordingnbr,volume,page,platnbr,plattype,...,propertytype,principaluse,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning
4,3027422,213043,120,2019-12-20,560000,20191226000848,,,,,...,11,6,3,N,N,N,N,1,8,
118,2999169,919715,200,2019-07-08,192000,20190712001080,,,,,...,3,2,3,N,N,N,N,1,3,
144,3000673,894444,200,2019-06-26,185000,20190722001395,,,,,...,3,2,3,N,N,N,N,1,3,
164,3002257,940652,630,2019-07-22,435000,20190730001339,,,,,...,11,6,3,N,N,N,N,1,8,
445,2980836,937630,695,2019-03-28,550000,20190404001008,,,,,...,3,6,3,N,N,N,N,1,8,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2089056,3004408,066000,2210,2019-08-07,41040000,20190812000849,,,,,...,3,7,22,N,N,N,N,18,2,45
2089057,3004408,066000,2225,2019-08-07,41040000,20190812000849,,,,,...,3,7,22,N,N,N,N,18,2,45
2089058,3004408,066000,2195,2019-08-07,41040000,20190812000849,,,,,...,3,7,22,N,N,N,N,18,2,45
2089059,3004408,066000,2220,2019-08-07,41040000,20190812000849,,,,,...,3,7,22,N,N,N,N,18,2,45


In [8]:
Real_Prop19['minor'] = Real_Prop19['minor'].astype(str)

<ipython-input-8-f11f99b810dc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Real_Prop19['minor'] = Real_Prop19['minor'].astype(str)


In [9]:
def minor_padded(row):
    if len(row) == 4:
        return row
    elif len(row) == 3: 
        return '0'+row
    elif len(row) == 2:
        return '00'+row
    elif len(row) == 1:
        return '000'+row
    else:
        return '0000'
Real_Prop19['minor'] = Real_Prop19.minor.apply(minor_padded)

<ipython-input-9-4ebf1bfcee56>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Real_Prop19['minor'] = Real_Prop19.minor.apply(minor_padded)


In [10]:
Real_Prop19

,excisetaxnbr,major,minor,documentdate,saleprice,recordingnbr,volume,page,platnbr,plattype,...,propertytype,principaluse,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning
4,3027422,213043,0120,2019-12-20,560000,20191226000848,,,,,...,11,6,3,N,N,N,N,1,8,
118,2999169,919715,0200,2019-07-08,192000,20190712001080,,,,,...,3,2,3,N,N,N,N,1,3,
144,3000673,894444,0200,2019-06-26,185000,20190722001395,,,,,...,3,2,3,N,N,N,N,1,3,
164,3002257,940652,0630,2019-07-22,435000,20190730001339,,,,,...,11,6,3,N,N,N,N,1,8,
445,2980836,937630,0695,2019-03-28,550000,20190404001008,,,,,...,3,6,3,N,N,N,N,1,8,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2089056,3004408,066000,2210,2019-08-07,41040000,20190812000849,,,,,...,3,7,22,N,N,N,N,18,2,45
2089057,3004408,066000,2225,2019-08-07,41040000,20190812000849,,,,,...,3,7,22,N,N,N,N,18,2,45
2089058,3004408,066000,2195,2019-08-07,41040000,20190812000849,,,,,...,3,7,22,N,N,N,N,18,2,45
2089059,3004408,066000,2220,2019-08-07,41040000,20190812000849,,,,,...,3,7,22,N,N,N,N,18,2,45


In [11]:
Real_Prop19['major_minor'] = Real_Prop19['major']+Real_Prop19['minor']

<ipython-input-11-5f8e4193e77f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Real_Prop19['major_minor'] = Real_Prop19['major']+Real_Prop19['minor']


In [12]:
Real_Prop19

,excisetaxnbr,major,minor,documentdate,saleprice,recordingnbr,volume,page,platnbr,plattype,...,principaluse,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning,major_minor
4,3027422,213043,0120,2019-12-20,560000,20191226000848,,,,,...,6,3,N,N,N,N,1,8,,2130430120
118,2999169,919715,0200,2019-07-08,192000,20190712001080,,,,,...,2,3,N,N,N,N,1,3,,9197150200
144,3000673,894444,0200,2019-06-26,185000,20190722001395,,,,,...,2,3,N,N,N,N,1,3,,8944440200
164,3002257,940652,0630,2019-07-22,435000,20190730001339,,,,,...,6,3,N,N,N,N,1,8,,9406520630
445,2980836,937630,0695,2019-03-28,550000,20190404001008,,,,,...,6,3,N,N,N,N,1,8,,9376300695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2089056,3004408,066000,2210,2019-08-07,41040000,20190812000849,,,,,...,7,22,N,N,N,N,18,2,45,0660002210
2089057,3004408,066000,2225,2019-08-07,41040000,20190812000849,,,,,...,7,22,N,N,N,N,18,2,45,0660002225
2089058,3004408,066000,2195,2019-08-07,41040000,20190812000849,,,,,...,7,22,N,N,N,N,18,2,45,0660002195
2089059,3004408,066000,2220,2019-08-07,41040000,20190812000849,,,,,...,7,22,N,N,N,N,18,2,45,0660002220


In [13]:
col_name = 'major_minor'
last_col = Real_Prop19.pop(col_name)
Real_Prop19.insert(1, col_name, last_col)

### Cleaning up Residential Buildings dataframe ###

In [14]:
Res_Bldg.rename(columns=str.lower, inplace=True)

In [15]:
Res_Bldg['major'] = Res_Bldg['major'].astype(str)
Res_Bldg['minor'] = Res_Bldg['minor'].astype(str)

In [16]:
Res_Bldg['major'] = Res_Bldg['major'].apply(major_padded)
Res_Bldg['minor'] = Res_Bldg['minor'].apply(minor_padded)

In [17]:
Res_Bldg['major_minor'] = Res_Bldg['major']+Res_Bldg['minor']

In [18]:
col_name = 'major_minor'
last_col = Res_Bldg.pop(col_name)
Res_Bldg.insert(0, col_name, last_col)

In [19]:
Res_Bldg.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515263 entries, 0 to 515262
Data columns (total 51 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   major_minor         515263 non-null  object 
 1   major               515263 non-null  object 
 2   minor               515263 non-null  object 
 3   bldgnbr             515263 non-null  int64  
 4   nbrlivingunits      515263 non-null  int64  
 5   address             515263 non-null  object 
 6   buildingnumber      515263 non-null  object 
 7   fraction            515263 non-null  object 
 8   directionprefix     514721 non-null  object 
 9   streetname          515263 non-null  object 
 10  streettype          515263 non-null  object 
 11  directionsuffix     514721 non-null  object 
 12  zipcode             469778 non-null  object 
 13  stories             515263 non-null  float64
 14  bldggrade           515263 non-null  int64  
 15  bldggradevar        515263 non-nul

### Cleaning up Parcel info ###

In [20]:
Parcel.rename(columns=str.lower, inplace=True)

In [21]:
Parcel['major'] = Parcel['major'].astype(str)
Parcel['minor'] = Parcel['minor'].astype(str)

In [22]:
Parcel['major'] = Parcel['major'].apply(major_padded)
Parcel['minor'] = Parcel['minor'].apply(minor_padded)

In [23]:
Parcel['major_minor'] = Parcel['major']+Parcel['minor']

In [24]:
last_col = Parcel.pop(col_name)
Parcel.insert(0, col_name, last_col)

In [25]:
Parcel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614843 entries, 0 to 614842
Data columns (total 82 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   major_minor             614843 non-null  object 
 1   major                   614843 non-null  object 
 2   minor                   614843 non-null  object 
 3   propname                601757 non-null  object 
 4   platname                520495 non-null  object 
 5   platlot                 614843 non-null  object 
 6   platblock               614843 non-null  object 
 7   range                   614843 non-null  int64  
 8   township                614843 non-null  int64  
 9   section                 614843 non-null  int64  
 10  quartersection          614843 non-null  object 
 11  proptype                614843 non-null  object 
 12  area                    614813 non-null  float64
 13  subarea                 614813 non-null  float64
 14  specarea            

**Curious to see what it all looks like together**

Merging together Real Property Sales from 2019 and Residential Buildings dataframe on their major_minor pin

In [26]:
prop_res = Real_Prop19.merge(Res_Bldg, on='major_minor', how='inner', suffixes=('', '_res' ))

In [27]:
res_rp = Res_Bldg.merge(Real_Prop19, on='major_minor', suffixes=("", "_rp"))

Lost many entries need to investigate and see what really happened on these merges.....

Next steps: what's the right way to get them together? what to do with the duplicates?